# Create Container to Generate Data

In [1]:
%%bash
mkdir -p ./daily_generate
cat > ./daily_generate/generatedata.py <<CODE

import os
import datetime

BaseDay=datetime.datetime(2022, 6, 15, 1, 58, 45, 476805)
Today=datetime.datetime.now()
diff=(Today-BaseDay).days
ds=str(Today)[:10]

model_dir='gs://seangoh-smu-mle-usa/Models/FaceMaskEfficientNetModel'
old_data_dir='gs://seangoh-smu-mle-usa/FaceMask/Test'
drift_data_dir='gs://seangoh-smu-mle-usa/FaceMask/Drift'
#os.system('mkdir -p ./app')
os.system('mkdir -p /app/{}/WithMask'.format(ds))
os.system('mkdir -p /app/{}/WithoutMask'.format(ds))
os.system('gsutil -mq cp -r {} /app'.format(model_dir)) #remove . for docker image
os.system('gsutil -mq cp -r {} /app'.format(old_data_dir))
os.system('gsutil -mq cp -r {} /app'.format(drift_data_dir))

DriftMaskDir='/app/Drift/WithMask'
DriftNoMaskDir='/app/Drift/WithoutMask'
OriginalMaskDir='/app/Test/WithMask'
OriginalNoMaskDir='/app/Test/WithoutMask'
DestinationWithMask='/app/{}/WithMask'.format(ds)
DestinationWithoutMask='/app/{}/WithoutMask'.format(ds)
CopyFrom='/app/{}'.format(ds)

import random
#Change Period Elapsed to Variable
PeriodElapsed=diff
DataDecayRatio=0.9
DataSizeTrend=1.1
DistributionDrift=0.9
BaseTotalSample=100
TotalSample=BaseTotalSample*(0.7+0.6*(random.random())*DataSizeTrend**PeriodElapsed)
CurrentDriftRatio=1-DataDecayRatio**(PeriodElapsed+random.random()/2)
CurrentWithoutMaskRatio=1-0.5*DistributionDrift**(PeriodElapsed-1+random.random()/2)

DriftSampleSize=int(TotalSample*CurrentDriftRatio)
OriginalSampleSize=int(TotalSample-DriftSampleSize)

DriftWithoutMask=int(CurrentWithoutMaskRatio*DriftSampleSize)
DriftWithMask=int(DriftSampleSize-DriftWithoutMask)
OriginalWithoutMask=int(CurrentWithoutMaskRatio*OriginalSampleSize)
OriginalWithMask=int(OriginalSampleSize-OriginalWithoutMask)



DriftMaskNum=len(os.listdir(DriftMaskDir))
DriftNoMaskNum=len(os.listdir(DriftNoMaskDir))
OriginalMaskNum=len(os.listdir(OriginalMaskDir))
OriginalNoMaskNum=len(os.listdir(OriginalNoMaskDir))

DMSample,DMRepeat=DriftWithMask%DriftMaskNum,DriftWithMask//DriftMaskNum
DNMSample,DNMRepeat=DriftWithoutMask%DriftNoMaskNum,DriftWithoutMask//DriftNoMaskNum
OMSample,OMRepeat=OriginalWithMask%OriginalMaskNum,OriginalWithMask//OriginalMaskNum
ONMSample,ONMRepeat=OriginalWithoutMask%OriginalNoMaskNum,OriginalWithoutMask//OriginalNoMaskNum

def GeneratePaths(datadir,repeat,sample):
    dirs=os.listdir(datadir)
    paths=[os.path.join(datadir,imagedir) for imagedir in dirs*repeat]
    sampleindex=random.sample(dirs,sample)
    newpaths=[os.path.join(datadir,i) for i in sampleindex]
    paths.extend(newpaths)
    return paths

#Generate Directories to Copy
DriftMaskPaths=GeneratePaths(DriftMaskDir,DMRepeat,DMSample)
DriftNoMaskPaths=GeneratePaths(DriftNoMaskDir,DNMRepeat,DNMSample)
OriginalMaskPaths=GeneratePaths(OriginalMaskDir,OMRepeat,OMSample)
OriginalNoMaskPaths=GeneratePaths(OriginalNoMaskDir,ONMRepeat,ONMSample)

#Copy Files
import shutil
def GenerateFiles(filepaths,folderdestination,prefix):
    for num,path in enumerate(filepaths):
        destination=os.path.join(folderdestination,prefix+str(num)+'.jpeg')
        shutil.copyfile(path,destination)
    return


GenerateFiles(DriftMaskPaths,DestinationWithMask,'D')
GenerateFiles(DriftNoMaskPaths,DestinationWithoutMask,'DN')
GenerateFiles(OriginalMaskPaths,DestinationWithMask,'O')
GenerateFiles(OriginalNoMaskPaths,DestinationWithoutMask,'ON')

#os.system('gsutil rm gs://seangoh-smu-mle-usa/DailyQC/{}'.format(ds))
shellcommand='gsutil -mq cp -r {} gs://seangoh-smu-mle-usa/DailyQC/'.format(CopyFrom,ds)
os.system(shellcommand)
CODE

Create Docker File and Image

In [2]:
%%bash

cat > ./daily_generate/Dockerfile <<EOF
FROM gcr.io/deeplearning-platform-release/base-cpu
WORKDIR /app

COPY . /app

ENTRYPOINT ["python", "generatedata.py"]
EOF

In [3]:
%%bash
docker build ./daily_generate/ -t masketeers/generatedata

Sending build context to Docker daemon  6.144kB
Step 1/4 : FROM gcr.io/deeplearning-platform-release/base-cpu
 ---> 4d10005d4e6f
Step 2/4 : WORKDIR /app
 ---> Using cache
 ---> f92293ea174a
Step 3/4 : COPY . /app
 ---> Using cache
 ---> ec9141899947
Step 4/4 : ENTRYPOINT ["python", "generatedata.py"]
 ---> Using cache
 ---> 14debabcc583
Successfully built 14debabcc583
Successfully tagged masketeers/generatedata:latest


In [4]:
!docker run masketeers/generatedata

Check that data is successfully created

In [5]:
import datetime
Today=datetime.datetime.now()
ds=str(Today)[:10]

!gsutil ls gs://seangoh-smu-mle-usa/DailyQC
!gsutil ls gs://seangoh-smu-mle-usa/DailyQC/$ds

gs://seangoh-smu-mle-usa/DailyQC/2022-06-15/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-16/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-17/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-18/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-19/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-20/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-21/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-22/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-23/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-24/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-25/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-26/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-27/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-27/WithMask/
gs://seangoh-smu-mle-usa/DailyQC/2022-06-27/WithoutMask/


In [6]:
!docker build ./daily_generate/ -t us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/generatedata:latest
!docker push us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/generatedata:latest

Sending build context to Docker daemon  6.144kB
Step 1/4 : FROM gcr.io/deeplearning-platform-release/base-cpu
 ---> 4d10005d4e6f
Step 2/4 : WORKDIR /app
 ---> Using cache
 ---> f92293ea174a
Step 3/4 : COPY . /app
 ---> Using cache
 ---> ec9141899947
Step 4/4 : ENTRYPOINT ["python", "generatedata.py"]
 ---> Using cache
 ---> 14debabcc583
Successfully built 14debabcc583
Successfully tagged us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/generatedata:latest
The push refers to repository [us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/generatedata]

47bfb344: Preparing 
6e8edcb8: Preparing 
37ed3ebe: Preparing 
0c97e748: Preparing 
22d8d85c: Preparing 
b01c5179: Preparing 
e696ff5b: Preparing 
43fff4a9: Preparing 
9c14a32d: Preparing 
d198ee7d: Preparing 
46a1d19e: Preparing 
b83c1c63: Preparing 
c31d7bf8: Preparing 
03bffb47: Preparing 
d4d6cdf8: Preparing 
bf18a086: Preparing 
c4cf558f: Preparing 
bf18a086: Preparing 
6f75faab: Preparing 
edc62fb3: Layer already exists lat

# Daily Prediction

In [7]:
%%bash

mkdir -p ./daily_predict

cat > ./daily_predict/daily_predict.py <<CODE

import tensorflow as tf
from tensorflow import keras
import numpy as np
#from PIL import Image
import os
from tensorflow.keras import layers
import datetime
#get daily directory name
Today=datetime.datetime.now()
ds=str(Today)[:10]
dailydatadir='gs://seangoh-smu-mle-usa/DailyQC/{}'.format(ds)


#download data
os.system('gsutil -mq cp -r {} /app/'.format(dailydatadir))
#download model
os.system('gsutil cp -r gs://seangoh-smu-mle-usa/Models/FaceMaskEfficientNetModel /app/')

dailydirectory="/app/{}/".format(ds)
image_size=224
ValidationData=keras.utils.image_dataset_from_directory(dailydirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = keras.models.load_model('FaceMaskEfficientNetModel')
          
import numpy as np
all_label=[]
all_pred=[]
for data,labels in ValidationData:
    all_label.extend(list(labels))
    probs=model.predict(data)
    preds=np.argmax(probs, axis=1)
    all_pred.extend(list(preds))
          
from sklearn.metrics import f1_score,accuracy_score
f1=float(f1_score(all_pred,all_label))
accuracy=float(accuracy_score(all_pred,all_label))
predict_1=int(np.sum(all_pred))
predict_0=int(np.sum([1 for i in all_pred if i==0]))
label_1=int(np.sum(all_label))
label_0=int(np.sum([1 for i in all_label if i==0]))
predict_1_percent=float(predict_1/(predict_0+predict_1))
predict_0_percent=float(predict_0/(predict_0+predict_1))
label_1_percent=float(label_1/(label_0+label_1))
label_0_percent=float(label_0/(label_0+label_1))

#creat json
#dailyperformance={'accuracy':accuracy, 'f1':f1}
#import json
#with open('dailyperformance.json', 'w') as f:
#    json.dump(dailyperformance, f)
#os.system('gsutil cp dailyperformance.json gs://seangoh-smu-mle-usa/testupload/')
    
#upload bq    
os.system('gcloud auth activate-service-account --key-file=daring-hash-348101-84e938ac4698.json')
print("prediction_finished")
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    'daring-hash-348101-84e938ac4698.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
dataset_id='facemask'
schematable_id='dailyqc_results'
table_ref=client.dataset(dataset_id).table(schematable_id)
table=client.get_table(table_ref)
row_insert=[(ds,f1,accuracy,predict_1,predict_0,label_1,label_0,predict_1_percent,predict_0_percent,label_1_percent,label_0_percent)]
client.insert_rows(table,row_insert)


CODE

In [8]:
%%bash

cat > ./daily_predict/Dockerfile <<EOF
FROM gcr.io/deeplearning-platform-release/tf-gpu.2-8
WORKDIR /app

COPY . /app

ENTRYPOINT ["python", "daily_predict.py"]
EOF


In [9]:
%%bash
cp daring-hash-348101-84e938ac4698.json ./daily_predict

In [10]:
%%bash
docker build ./daily_predict/ -t masketeers/dailypredict

Sending build context to Docker daemon  15.87kB
Step 1/4 : FROM gcr.io/deeplearning-platform-release/tf-gpu.2-8
 ---> cc037125fdd9
Step 2/4 : WORKDIR /app
 ---> Using cache
 ---> 884c3492a942
Step 3/4 : COPY . /app
 ---> 9c273fe9274e
Step 4/4 : ENTRYPOINT ["python", "daily_predict.py"]
 ---> Running in fe73fc405ae8
Removing intermediate container fe73fc405ae8
 ---> 7db12a193d8e
Successfully built 7db12a193d8e
Successfully tagged masketeers/dailypredict:latest


In [11]:
!docker run masketeers/dailypredict

Copying gs://seangoh-smu-mle-usa/Models/FaceMaskEfficientNetModel/keras_metadata.pb...
Copying gs://seangoh-smu-mle-usa/Models/FaceMaskEfficientNetModel/saved_model.pb...
Copying gs://seangoh-smu-mle-usa/Models/FaceMaskEfficientNetModel/variables/variables.data-00000-of-00001...
Copying gs://seangoh-smu-mle-usa/Models/FaceMaskEfficientNetModel/variables/variables.index...
/ [4 files][ 20.5 MiB/ 20.5 MiB]                                                
Operation completed over 4 objects/20.5 MiB.                                     
2022-06-27 14:07:26.629864: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-06-27 14:07:26.629962: W tensorflow/stream_executor/c

In [12]:
!docker build ./daily_predict/ -t us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/dailypredict:latest
!docker push us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/dailypredict:latest

Sending build context to Docker daemon  15.87kB
Step 1/4 : FROM gcr.io/deeplearning-platform-release/tf-gpu.2-8
 ---> cc037125fdd9
Step 2/4 : WORKDIR /app
 ---> Using cache
 ---> 884c3492a942
Step 3/4 : COPY . /app
 ---> Using cache
 ---> 9c273fe9274e
Step 4/4 : ENTRYPOINT ["python", "daily_predict.py"]
 ---> Using cache
 ---> 7db12a193d8e
Successfully built 7db12a193d8e
Successfully tagged us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/dailypredict:latest
The push refers to repository [us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/dailypredict]

ff899eec: Preparing 
95a9cd9c: Preparing 
b3120056: Preparing 
53b734c4: Preparing 
b4a993bf: Preparing 
a0734f1b: Preparing 
39fb5680: Preparing 
6da164cd: Preparing 
22d8d85c: Preparing 
b01c5179: Preparing 
e696ff5b: Preparing 
43fff4a9: Preparing 
86de6044: Preparing 
8d193daf: Preparing 
188023c9: Preparing 
0496c2b3: Preparing 
dc387b12: Preparing 
257dc2e4: Preparing 
54032850: Preparing 
951137ff: Preparing 
c25e1d03: 

In [13]:
#!gsutil rm -r gs://seangoh-smu-mle-usa/DailyQC/$ds

# Create Pipeline

In [14]:
import kfp
import google.cloud.aiplatform as aip
from kfp.v2 import compiler
from kfp.v2.dsl import (
    Dataset, Model, Output, Input, 
    OutputPath, ClassificationMetrics, Metrics, component
)
import kfp.v2
from google_cloud_pipeline_components import aiplatform as gcc_aip

DISPLAY_NAME='facemask'
PROJECT_ID='daring-hash-348101'
REGION='us-east1'
GCS_BUCKET='seangoh-smu-mle-usa'
PIPELINE_ROOT='gs://seangoh-smu-mle-usa/projectpipeline'
BUCKET_URI='gs://'+GCS_BUCKET

@kfp.v2.dsl.pipeline(name=DISPLAY_NAME,pipeline_root=PIPELINE_ROOT)
def pipeline(project: str = PROJECT_ID, 
    region: str = REGION, bucket:str = GCS_BUCKET):

    generate_data=gcc_aip.CustomContainerTrainingJobRunOp(
        display_name=DISPLAY_NAME,
        container_uri='us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/generatedata:latest',
        project=project,
        location=region,
        staging_bucket=bucket,
        machine_type="n1-standard-4",
        replica_count=1, 
    ).set_caching_options(False)

    evaluate_upload=gcc_aip.CustomContainerTrainingJobRunOp(
        display_name=DISPLAY_NAME,
        container_uri='us-east1-docker.pkg.dev/daring-hash-348101/smu-mle-usa/dailypredict',
        project=project,
        location=region,
        staging_bucket=bucket,
        machine_type="n1-standard-4",
        #accelerator_type='NVIDIA_TESLA_K80',
        replica_count=1, 
    ).after(generate_data).set_caching_options(False)
    

In [15]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="dailypredict.json",
)


import google.cloud.aiplatform as aip
job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="dailypredict.json",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
    location='us-east1'
)
job.run()

Creating PipelineJob


/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


PipelineJob created. Resource name: projects/591661299323/locations/us-east1/pipelineJobs/facemask-20220627140831
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/591661299323/locations/us-east1/pipelineJobs/facemask-20220627140831')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-east1/pipelines/runs/facemask-20220627140831?project=591661299323
PipelineJob projects/591661299323/locations/us-east1/pipelineJobs/facemask-20220627140831 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/591661299323/locations/us-east1/pipelineJobs/facemask-20220627140831 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/591661299323/locations/us-east1/pipelineJobs/facemask-20220627140831 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/591661299323/locations/us-east1/pipelineJobs/facemask-20220627140831 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJo

In [16]:
!gsutil cp dailypredict.json gs://seangoh-smu-mle-usa/ProjectPipeline/

Copying file://dailypredict.json [Content-Type=application/json]...
/ [1 files][  6.1 KiB/  6.1 KiB]                                                
Operation completed over 1 objects/6.1 KiB.                                      


In [17]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(project_id=PROJECT_ID, region=REGION)
    
response = api_client.create_schedule_from_job_spec(
    job_spec_path="gs://seangoh-smu-mle-usa/ProjectPipeline/dailypredict.json",
    schedule="15 15 * * *",
    time_zone="America/Indiana/Knox",  # change this as necessary
    parameter_values={'project':PROJECT_ID, 'region':REGION, 'bucket': GCS_BUCKET},
    #pipeline_root=PIPELINE_ROOT,  # this argument is necessary if you did not specify PIPELINE_ROOT as part of the pipeline definition.
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v2.0.0. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,
